In [1]:
import pandas as pd
# df = pd.read_csv("concatenated_results_Paragraph_title_COUNTERFACTUAL.csv")
df = pd.read_csv("concatenated_results_Paragraph_summary_COUNTERFACTUAL.csv")

In [2]:
df

,model_1_gold_lp_median,model_1_gold_lp_max,model_1_gold_lp_min,model_1_gold_lp_std,model_1_gold_lp_moment1,model_1_gold_lp_moment2,model_1_gold_lp_moment3,model_1_gold_lp_moment4,model_1_gold_lp_moment5,model_1_gold_lp_q95,...,kl_4_vs_6_q80,kl_4_vs_6_q20,kl_4_vs_6_q15,kl_4_vs_6_q10,kl_4_vs_6_q05,label,source_file,file_path,counterfactual,Final_label
0,0.000000e+00,0.0,-0.469582,0.062336,-0.009575,0.003819,-0.001683,0.000772,-0.000355,0.0,...,0.019408,6.957359e-07,3.180618e-07,2.052192e-07,1.287433e-08,fact,logprob_matrix_1.csv,NCT00001959/logprob_matrix_1.csv,fact,fact
1,-2.980232e-07,0.0,-11.196398,2.163222,-0.524352,4.512401,-43.779671,464.517646,-4946.832018,0.0,...,0.254785,7.169600e-05,2.824224e-05,1.488498e-05,3.977073e-06,fact,logprob_matrix_10.csv,NCT00001959/logprob_matrix_10.csv,fact,fact
2,0.000000e+00,0.0,-1.318674,0.358092,-0.183229,0.119681,-0.096271,0.112248,-0.125854,0.0,...,0.536783,1.409192e-04,1.123663e-04,8.786914e-05,7.300707e-05,fact,logprob_matrix_11.csv,NCT00001959/logprob_matrix_11.csv,fact,fact
3,0.000000e+00,0.0,-7.702140,0.904958,-0.141555,0.807731,-5.921203,44.762865,-338.419368,0.0,...,0.195337,1.136319e-06,6.168359e-07,2.115952e-07,2.655602e-08,fact,logprob_matrix_12.csv,NCT00001959/logprob_matrix_12.csv,fact,fact
4,-5.513278e-05,0.0,-6.755051,0.421117,-0.165742,0.176993,-0.619950,3.746766,-24.333245,0.0,...,0.163484,6.523635e-05,1.996456e-05,7.435319e-06,2.005330e-06,fact,logprob_matrix_13.csv,NCT00001959/logprob_matrix_13.csv,fact,fact
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,-1.876774e-04,0.0,-5.402549,0.386656,-0.136843,0.149211,-0.386540,1.657368,-8.167006,0.0,...,0.249763,1.441484e-04,4.974966e-05,5.792682e-06,6.281551e-07,fact,logprob_matrix_5.csv,NCT04739800/logprob_matrix_5.csv,fact,fact
1496,-1.329166e-05,0.0,-10.257800,3.241725,-1.031837,9.457901,-77.559497,725.509797,-6683.318437,0.0,...,1.421351,1.745008e-04,8.185911e-05,9.656293e-06,8.989429e-06,fact,logprob_matrix_6.csv,NCT04739800/logprob_matrix_6.csv,fact,fact
1497,0.000000e+00,0.0,-6.091068,1.194335,-0.253525,1.371574,-7.636523,44.666768,-260.720646,0.0,...,0.039108,5.195794e-05,2.996927e-05,1.289915e-05,6.302248e-06,fact,logprob_matrix_7.csv,NCT04739800/logprob_matrix_7.csv,fact,fact
1498,0.000000e+00,0.0,-6.523907,1.152263,-0.226127,1.286219,-7.795517,49.161001,-309.589898,0.0,...,0.119939,2.643177e-05,2.159779e-05,3.763596e-06,5.721547e-07,fact,logprob_matrix_8.csv,NCT04739800/logprob_matrix_8.csv,NaN,NaN


In [3]:
import pandas as pd
import os


files_to_process = [
            'NCT00001959.json', 
            'NCT00448136.json', 
            'NCT00726986.json', 
            'NCT01226485.json',
            'NCT01334723.json',
            'NCT01338987.json',
            'NCT01696968.json',
            'NCT02684058.json',
            'NCT03158220.json',
            'NCT03271047.json'
        ]

# Assuming your DataFrame is called df
filtered_df = pd.DataFrame()

# Filter rows where NCT number is in files_to_process
for file in files_to_process:
    nct_number = file.replace('.json', '')
    filtered_rows = df[df['file_path'].str.contains(nct_number)]
    filtered_df = pd.concat([filtered_df, filtered_rows])

# Reset index of the final filtered DataFrame
filtered_df = filtered_df.reset_index(drop=True)

# Add new column for GPT labels
filtered_df['label_gpt'] = None

# Path to the folder containing the JSON files with GPT evaluations
# json_folder = "/home/4481281/Clinical_trials/Original_format/Results/Factuality_Eval/Eval_vs_Summary/Paragraph_level/Evaluation_title_Paragraph_GPT"
json_folder = "/home/4481281/Clinical_trials/Original_format/Results/Factuality_Eval/Eval_vs_Summary/Paragraph_level/Evaluation_summary_Paragraph_GPT"

# Iterate through each row in filtered_df
for idx, row in filtered_df.iterrows():
    try:
        # Extract NCT number and question number
        nct_number = row['file_path'].split('/')[0]  # Gets NCT00001959 from path
        question_num = row['source_file'].split('_')[-1].replace('.csv', '')  # Gets number from logprob_matrix_11.csv
        
        # Construct JSON filename and path
        json_filename = f"{nct_number}.json"
        json_path = os.path.join(json_folder, json_filename)
        
        # Read JSON file
        if os.path.exists(json_path):
            with open(json_path, 'r') as f:
                gpt_data = json.load(f)
            
            # Get label for this question
            question_key = f"Q{question_num}"
            if question_key in gpt_data:
                 # Map GPT's yes/no to fact/hallucination
                gpt_label = gpt_data[question_key]['label'].lower()
                if gpt_label == 'yes':
                    filtered_df.at[idx, 'label_gpt'] = 'fact'
                elif gpt_label == 'no':
                    filtered_df.at[idx, 'label_gpt'] = 'hallucination'
                else:
                    filtered_df.at[idx, 'label_gpt'] = 'N/A'
            else:
                print(f"Question {question_key} not found in {json_filename}")
                filtered_df.at[idx, 'label_gpt'] = 'N/A'
        else:
            print(f"File not found: {json_path}")
            filtered_df.at[idx, 'label_gpt'] = 'N/A'
            
    except Exception as e:
        print(f"Error processing row {idx}: {str(e)}")
        filtered_df.at[idx, 'label_gpt'] = 'N/A'


# Save updated DataFrame
#filtered_df.to_csv('testset_title.csv', index=False)
filtered_df.to_csv('testset_summary.csv', index=False)


# Print summary
print("\nSummary:")
print(f"Total rows processed: {len(filtered_df)}")
print("\nGPT label distribution:")
print(filtered_df['label_gpt'].value_counts())

# Print agreement with original labels
print("\nAgreement with original labels:")
agreement = (filtered_df['label'] == filtered_df['label_gpt']).mean() * 100
print(f"Agreement percentage: {agreement:.2f}%")


Summary:
Total rows processed: 150

GPT label distribution:
label_gpt
fact             144
hallucination      6
Name: count, dtype: int64

Agreement with original labels:
Agreement percentage: 95.33%


## Add counterfactual

In [4]:
import pandas as pd
import os


# Reset index of the final filtered DataFrame
#filtered_df = pd.read_csv("testset_title.csv")
filtered_df = pd.read_csv("testset_summary.csv")


# Add new column for GPT labels
filtered_df['counterfactual_gpt'] = None

# Path to the folder containing the JSON files with GPT evaluations
# json_folder = "/home/4481281/Clinical_trials/Original_format/Results/Factuality_Eval/Eval_vs_Summary_Counterfactual/Paragraph_level/Evaluation_title_Paragraph_GPT"
json_folder = "/home/4481281/Clinical_trials/Original_format/Results/Factuality_Eval/Eval_vs_Summary_Counterfactual/Paragraph_level/Evaluation_summary_Paragraph_GPT"

# Iterate through each row in filtered_df
for idx, row in filtered_df.iterrows():
    try:
        # Extract NCT number and question number
        nct_number = row['file_path'].split('/')[0]  # Gets NCT00001959 from path
        question_num = row['source_file'].split('_')[-1].replace('.csv', '')  # Gets number from logprob_matrix_11.csv
        
        # Construct JSON filename and path
        json_filename = f"{nct_number}.json"
        json_path = os.path.join(json_folder, json_filename)
        
        # Read JSON file
        if os.path.exists(json_path):
            with open(json_path, 'r') as f:
                gpt_data = json.load(f)
            
            # Get label for this question
            question_key = f"Q{question_num}"
            if question_key in gpt_data:
                 # Map GPT's yes/no to fact/hallucination
                gpt_label = gpt_data[question_key]['label'].lower()
                if gpt_label == 'yes':
                    filtered_df.at[idx, 'counterfactual_gpt'] = 'hallucination'
                elif gpt_label == 'no':
                    filtered_df.at[idx, 'counterfactual_gpt'] = 'fact'
                else:
                    filtered_df.at[idx, 'counterfactual_gpt'] = 'N/A'
            else:
                print(f"Question {question_key} not found in {json_filename}")
                filtered_df.at[idx, 'label_gpt'] = 'N/A'
        else:
            print(f"File not found: {json_path}")
            filtered_df.at[idx, 'counterfactual_gpt'] = 'N/A'
            
    except Exception as e:
        print(f"Error processing row {idx}: {str(e)}")
        filtered_df.at[idx, 'counterfactual_gpt'] = 'N/A'


# Save updated DataFrame
# filtered_df.to_csv('testset_title_counterfactual.csv', index=False)
filtered_df.to_csv('testset_summary_counterfactual.csv', index=False)

# Print summary
print("\nSummary:")
print(f"Total rows processed: {len(filtered_df)}")
print("\nGPT label distribution:")
print(filtered_df['counterfactual_gpt'].value_counts())

# Print agreement with original labels
print("\nAgreement with original labels:")
agreement = (filtered_df['counterfactual'] == filtered_df['counterfactual_gpt']).mean() * 100
print(f"Agreement percentage: {agreement:.2f}%")


Summary:
Total rows processed: 150

GPT label distribution:
counterfactual_gpt
fact    150
Name: count, dtype: int64

Agreement with original labels:
Agreement percentage: 94.67%


## Add final_label with the overlap

In [5]:
import pandas as pd
import os
import numpy as np

# Reset index of the final filtered DataFrame
#df = pd.read_csv("testset_title_counterfactual.csv")
df = pd.read_csv("testset_summary_counterfactual.csv")


# Define conditions and choices for the new labels
conditions = [
            (df['label_gpt'] == 'fact') & (df['counterfactual_gpt'] == 'fact'),
            (df['label_gpt'] == 'hallucination') & (df['counterfactual_gpt'] == 'hallucination'),
            (df['label_gpt'] == 'fact') & (df['counterfactual_gpt'] == 'hallucination'),
            (df['label_gpt'] == 'hallucination') & (df['counterfactual_gpt'] == 'fact')
]
        
choices = ['fact', 'hallucination', 'error', 'coverage']

# Create new column with the conditions
df['Final_label_gpt'] = np.select(conditions, choices, default='N/A')

# Print summary of new labels
print("\nFinal Label Distribution:")
print(df['Final_label_gpt'].value_counts())

# Save updated DataFrame
#df.to_csv('testset_title_counterfactual-final.csv', index=False)
df.to_csv('testset_summary_counterfactual-final.csv', index=False)



Final Label Distribution:
Final_label_gpt
fact        144
coverage      6
Name: count, dtype: int64


## Concatenate TestSET

In [6]:
df1 = pd.read_csv("testset_title_counterfactual-final.csv")
df2 = pd.read_csv("testset_summary_counterfactual-final.csv")

# Concatenate the DataFrames
df_combined = pd.concat([df1, df2], axis=0, ignore_index=True)
# Print sample of combined data
print("\nSample of combined data:")

# Print value counts for labels if they exist
if 'Final_label_gpt' in df_combined.columns:
    print("\nLabel distribution:")
    print(df_combined['Final_label_gpt'].value_counts(normalize=True))

df_combined.to_csv('testset_final.csv', index=False)


Sample of combined data:

Label distribution:
Final_label_gpt
fact             0.600671
coverage         0.211409
hallucination    0.187919
Name: proportion, dtype: float64


In [7]:
# Calculate agreements for each category
fact_agreement = len(df_combined[(df_combined['Final_label_gpt'] == 'fact') & 
                               (df_combined['Final_label'] == 'fact')])

hallucination_agreement = len(df_combined[(df_combined['Final_label_gpt'] == 'hallucination') & 
                                        (df_combined['Final_label'] == 'hallucination')])

coverage_agreement = len(df_combined[(df_combined['Final_label_gpt'] == 'coverage') & 
                                   (df_combined['Final_label'] == 'coverage')])

# Print results
print("\nAgreement Analysis:")
print(f"Fact agreement: {fact_agreement} rows")
print(f"Hallucination agreement: {hallucination_agreement} rows")
print(f"Coverage agreement: {coverage_agreement} rows")

# Print total counts for each category in both columns for comparison
print("\nTotal counts in each column:")
print("\nFinal_label counts:")
print(df_combined['Final_label'].value_counts())
print("\nFinal_label_gpt counts:")
print(df_combined['Final_label_gpt'].value_counts())


Agreement Analysis:
Fact agreement: 167 rows
Hallucination agreement: 33 rows
Coverage agreement: 20 rows

Total counts in each column:

Final_label counts:
Final_label
fact             196
hallucination     40
coverage          32
error              5
Name: count, dtype: int64

Final_label_gpt counts:
Final_label_gpt
fact             179
coverage          63
hallucination     56
Name: count, dtype: int64
